In [ ]:
## File to generate all of the possible folding superpositions in the (2,2,2,...) case

In [56]:
## Makes a list of numbers 1 through 2n
def genLabels(n):
    labels = []
    for i in range(2*n):
        labels.append(i)
    return labels

## Makes poset edges for the (2,2,...) crease pattern
def genEdges2s(n):
    poset = []
    for i in range(2*n-1):
        if (i%4 == 0):
            poset.append([i, i+1])
        if (i%4 == 1):
            if (i+2 < 2*n):
                poset.append([i, i+2])
        if (i%4 == 2):
            poset.append([i+1, i])
        if (i%4 == 3):
            if (i+2 < 2*n):
                poset.append([i+2, i])
    return poset

def genPoset2s(n):
    return Poset((genLabels(n), genEdges2s(n)))

In [127]:
'''Code from other file'''

import itertools
import numpy as np

n = 8

adj = {}
for face in range(2 * n):
    adj[face] = []
    if face - 2 >= 0:
        adj[face].append(face-2)
    if face + 2 < n:
        adj[face].append(face+2)
    if face % 2 == 0:
        adj[face].append(face+1)
    if face % 2 == 1:
        adj[face].append(face-1)

bottom_edges = [(i, i+1) for i in range(0, 2 * n, 2)]
left_edges = [(i, i+2) for i in range(0, 2 * n - 2, 4)] + [(i, i+2) for i in range(1, 2 * n - 2, 4)]
right_edges = [(i, i+2) for i in range(2, 2 * n - 2, 4)] + [(i, i+2) for i in range(3, 2 * n - 2, 4)]

positive_faces = list(range(0, 2*n, 4)) + list(range(3, 2*n, 4))
negative_faces = list(range(2, 2*n, 4)) + list(range(1, 2*n, 4))

# check for crossing violations
def check_crossings(edges, ordering):
    for e1, e2 in itertools.combinations(edges, 2):
        b1 = min(ordering[e1[0]], ordering[e1[1]])
        t1 = max(ordering[e1[0]], ordering[e1[1]])
        b2 = min(ordering[e2[0]], ordering[e2[1]])
        t2 = max(ordering[e2[0]], ordering[e2[1]])

        if b1 < b2 < t1 < t2 or b2 < b1 < t2 < t1:
            # print("CROSSING", e1, e2, b1, t1, b2, t2)
            return False
    return True

# Check that MV is respected: faces with +1 parity (same as the top left) need to be below faces that they share a
# valley fold with, but faces with -1 parity need to satisfy the opposite conditions
def checkMV(faces, parity, assignment, ordering):
    for face in faces:
        for adj_ in adj[face]:
            edge = (min(adj_, face), max(adj_, face))
            # bunch of signs stuff lol
            if np.sign(ordering[face] - ordering[adj_]) != parity * assignment[edge]:
                # print("MV ERROR", face, adj)
                return False
    return True

def verify(assignment, ordering):

    if not check_crossings(bottom_edges, ordering) or not check_crossings(left_edges, ordering) or not check_crossings(right_edges, ordering):
        return False

    return True

def convertToMV(assign):
    edge_assign = {}
    edge_assign[(0,1)] = -1
    for i in range(len(assign)):
        top_left = 2*i
        base = edge_assign[(top_left, top_left+1)]
        if assign[i] == 1:
            edge_assign[(top_left, top_left+2)] = -1*base
            edge_assign[(top_left+1, top_left+3)] = -1*base
            edge_assign[(top_left+2, top_left+3)] = -1*base
        if assign[i] == 2:
            edge_assign[(top_left, top_left+2)] = base
            edge_assign[(top_left+1, top_left+3)] = -1*base
            edge_assign[(top_left+2, top_left+3)] = base
        if assign[i] == 3:
            edge_assign[(top_left, top_left+2)] = base
            edge_assign[(top_left+1, top_left+3)] = base
            edge_assign[(top_left+2, top_left+3)] = -1*base
        if assign[i] == 4:
            edge_assign[(top_left, top_left+2)] = -1*base
            edge_assign[(top_left+1, top_left+3)] = base
            edge_assign[(top_left+2, top_left+3)] = base
    return edge_assign

In [124]:
## Iterate over all possible linear extensions of the poset and check edges

def countFolds2s():
    assignment = convertToMV([2]*(n-1))
    count = 0
    P = genPoset2s(n)
    for i in P.linear_extension().parent():
        if(verify(assignment, np.argsort(i))):
            print(i)
            count+= 1
    
    return count

In [128]:
print(countFolds2s())

[0, 4, 5, 8, 9, 12, 13, 15, 14, 11, 10, 7, 6, 1, 3, 2]
[4, 5, 8, 9, 12, 13, 15, 14, 11, 10, 7, 6, 0, 1, 3, 2]
[0, 1, 4, 5, 8, 9, 12, 13, 15, 14, 11, 10, 7, 6, 3, 2]
[0, 4, 8, 9, 12, 13, 15, 14, 11, 10, 5, 1, 3, 7, 6, 2]
[0, 4, 5, 1, 3, 8, 9, 12, 13, 15, 14, 11, 10, 7, 6, 2]
[0, 4, 8, 12, 13, 9, 5, 1, 3, 7, 11, 15, 14, 10, 6, 2]
[0, 4, 8, 9, 5, 1, 3, 7, 12, 13, 15, 14, 11, 10, 6, 2]
[0, 4, 8, 12, 13, 15, 14, 9, 5, 1, 3, 7, 11, 10, 6, 2]
[12, 8, 9, 13, 15, 11, 4, 5, 0, 1, 3, 2, 7, 6, 10, 14]
[12, 8, 4, 5, 0, 1, 3, 2, 9, 13, 15, 11, 7, 6, 10, 14]
[12, 8, 9, 4, 5, 0, 1, 3, 2, 7, 6, 13, 15, 11, 10, 14]
[12, 13, 15, 8, 9, 4, 5, 0, 1, 3, 2, 7, 6, 11, 10, 14]
[12, 13, 8, 9, 4, 5, 0, 1, 3, 2, 7, 6, 11, 10, 15, 14]
[8, 9, 12, 13, 15, 14, 11, 10, 4, 5, 0, 1, 3, 2, 7, 6]
[4, 5, 0, 1, 3, 2, 8, 9, 12, 13, 15, 14, 11, 10, 7, 6]
[8, 9, 4, 5, 0, 1, 3, 2, 7, 6, 12, 13, 15, 14, 11, 10]
[12, 13, 15, 14, 8, 9, 4, 5, 0, 1, 3, 2, 7, 6, 11, 10]
[12, 8, 4, 0, 1, 5, 9, 13, 15, 11, 7, 3, 2, 6, 10, 14]
[12, 8, 4,